# 🏋️ LoRA Fine-Tuning: Mistral or TinyLLaMA on Financial Percentage QA
This notebook fine-tunes a model using PEFT + LoRA on a cleaned dataset of percentage-based questions.

In [ ]:
!pip install transformers datasets peft accelerate bitsandbytes -q

In [ ]:
import json
import re
import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from peft import get_peft_model, LoraConfig, TaskType
from transformers import Trainer, DataCollatorForLanguageModeling

In [ ]:
# Load cleaned dataset
with open("/mnt/data/dev_percent_cleaned.json") as f:
    data = json.load(f)

formatted = [{
    "text": f"You are a financial assistant.\nQuestion: {item['question']}\nAnswer: {item['answer']}"
} for item in data]

dataset = Dataset.from_list(formatted)
dataset = dataset.train_test_split(test_size=0.1)
dataset

In [ ]:
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # or use 'mistralai/Mistral-7B-v0.1'
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # 🛠 fix padding issue

def tokenize(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=256)

tokenized = dataset.map(tokenize, batched=True)
tokenized.set_format("torch", columns=["input_ids", "attention_mask"])
tokenized

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto"
)

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(base_model, lora_config)

In [ ]:
training_args = TrainingArguments(
    output_dir="./peft-finetuned-financial",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=2,
    evaluation_strategy="epoch",
    logging_dir="./logs",
    learning_rate=2e-4,
    fp16=torch.cuda.is_available(),
    save_strategy="epoch"
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

In [ ]:
trainer.train()

In [ ]:
model.save_pretrained("./peft-finetuned-financial")